# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder: OPTIONAL to execute C++ code or Rust code</h2>
            <span style="color:#f71;">As an alternative, you can run it on the website given yesterday</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">Important Note</h1>
            <span style="color:#900;">
            In this lab, I use high end models GPT 5, Claude 4.5 Sonnet, Gemini 2.5 Pro, Grok 4, which are the slightly higher priced models. The costs are still low, but if you'd prefer to keep costs ultra low, please pick lower cost models like gpt-5-nano.
            </span>
        </td>
    </tr>
</table>

In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess
from IPython.display import Markdown, display


In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:6]}")
else:
    print("OpenRouter API Key not set (and this is optional)")



OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI
Grok API Key not set (and this is optional)
Groq API Key exists and begins gsk_
OpenRouter API Key exists and begins sk-or-


In [3]:
# Connect to client libraries

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
grok_url = "https://api.x.ai/v1"
groq_url = "https://api.groq.com/openai/v1"
ollama_url = "http://localhost:11434/v1"
openrouter_url = "https://openrouter.ai/api/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
ollama = OpenAI(api_key="ollama", base_url=ollama_url)
openrouter = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)



In [4]:
models = ["gpt-5", "claude-sonnet-4-5-20250929", "grok-4", "gemini-2.5-pro", "qwen2.5-coder", "deepseek-coder-v2", "gpt-oss:20b", "qwen/qwen3-coder-30b-a3b-instruct", "openai/gpt-oss-120b", ]

clients = {"gpt-5": openai, "claude-sonnet-4-5-20250929": anthropic, "grok-4": grok, "gemini-2.5-pro": gemini, "openai/gpt-oss-120b": groq, "qwen2.5-coder": ollama, "deepseek-coder-v2": ollama, "gpt-oss:20b": ollama, "qwen/qwen3-coder-30b-a3b-instruct": openrouter}

# Want to keep costs ultra-low? Replace this with models of your choice, using the examples from yesterday

In [9]:
# Fix: Add Rust to PATH for the notebook kernel
import os
cargo_path = os.path.expanduser("~/.cargo/bin")
if cargo_path not in os.environ["PATH"]:
    os.environ["PATH"] = f"{cargo_path}:{os.environ['PATH']}"
print(f"Added {cargo_path} to PATH")

Added /home/sanjif/.cargo/bin to PATH


In [13]:
# Let's examine the notebook's PATH before we modified it
import os
import subprocess

# Get original PATH by checking what the parent process has
# (this won't show our modification since we already changed it)
current_path = os.environ["PATH"]

print("Current PATH in notebook (after our fix):")
for path in current_path.split(":"):
    print(f"  {path}")

# Check where this Python interpreter is
print(f"\nPython interpreter: {os.sys.executable}")

# Check parent process (VS Code)
print(f"\nParent PID: {os.getppid()}")

# The PATH came from VS Code's environment when it started this kernel
print("\n💡 The notebook's original PATH came from:")
print("   VS Code → Python kernel startup")
print("   (NOT from .bashrc)")

Current PATH in notebook (after our fix):
  /home/sanjif/.cargo/bin
  /home/sanjif/llm/.venv/bin
  /home/sanjif/.vscode-server/bin/585eba7c0c34fd6b30faac7c62a42050bfbc0086/bin/remote-cli
  /home/sanjif/.local/bin
  /home/sanjif/.nvm/versions/node/v24.12.0/bin
  /usr/local/sbin
  /usr/local/bin
  /usr/sbin
  /usr/bin
  /sbin
  /bin
  /usr/games
  /usr/local/games
  /usr/lib/wsl/lib
  /mnt/c/Program Files (x86)/NVIDIA Corporation/PhysX/Common
  /mnt/c/Windows/system32
  /mnt/c/Windows
  /mnt/c/Windows/System32/wbem
  /mnt/c/Windows/System32/WindowsPowerShell/v1.0
  /mnt/c/Windows/System32/OpenSSH
  /mnt/c/Program Files/Intel/WiFi/bin
  /mnt/c/Program Files/Common Files/Intel/WirelessCommon
  /mnt/c/Program Files/CMake/bin
  /mnt/c/Program Files/Microsoft SQL Server/Client SDK/ODBC/110/Tools/Binn
  /mnt/c/Program Files (x86)/Microsoft SQL Server/120/Tools/Binn
  /mnt/c/Program Files/Microsoft SQL Server/120/Tools/Binn
  /mnt/c/Program Files/Microsoft SQL Server/120/DTS/Binn
  /mnt/c/Progr

## Understanding Jupyter Notebook's Environment

Jupyter notebooks **don't inherit from `.bashrc`**! Here's the environment chain:

1. **VS Code** launches with its environment (from how you started VS Code - terminal, desktop launcher, etc.)
2. **Python kernel** is started by VS Code and inherits VS Code's environment
3. **`.bashrc` is never sourced** because it's only loaded by interactive bash shells

This is why we need to manually add Rust to PATH in the notebook using `os.environ["PATH"]`.

In [10]:
from system_info import retrieve_system_info, rust_toolchain_info

system_info = retrieve_system_info()
rust_info = rust_toolchain_info()
rust_info

{'installed': True,
 'rustc': {'path': '/home/sanjif/.cargo/bin/rustc',
  'version': 'rustc 1.92.0 (ded5c06cf 2025-12-08)',
  'host_triple': 'x86_64-unknown-linux-gnu',
  'release': '1.92.0',
  'commit_hash': 'ded5c06cf21d2b93bffd5d884aa6e96934ee4234'},
 'cargo': {'path': '/home/sanjif/.cargo/bin/cargo',
  'version': 'cargo 1.92.0 (344c4567c 2025-10-21)'},
 'rustup': {'path': '/home/sanjif/.cargo/bin/rustup',
  'version': 'rustup 1.28.2 (e4f3ad6f8 2025-04-28)',
  'active_toolchain': 'stable-x86_64-unknown-linux-gnu (default)',
  'default_toolchain': '',
  'toolchains': ['stable-x86_64-unknown-linux-gnu (active, default)'],
  'targets_installed': ['x86_64-unknown-linux-gnu']},
 'rust_analyzer': {'path': '/home/sanjif/.cargo/bin/rust-analyzer'},
 'env': {'CARGO_HOME': '/home/sanjif/.cargo',
  'RUSTUP_HOME': '/home/sanjif/.rustup',
  'RUSTFLAGS': '',
  'CARGO_BUILD_TARGET': ''},
 'execution_examples': ['"/home/sanjif/.cargo/bin/cargo" build',
  '"/home/sanjif/.cargo/bin/cargo" run',
  '"/

In [11]:
message = f"""
Here is a report of the system information for my computer.
I want to run a Rust compiler to compile a single rust file called main.rs and then execute it in the simplest way possible.
Please reply with whether I need to install a Rust toolchain to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile Rust code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.
Have the maximum possible runtime performance in mind; compile time can be slow. Fastest possible runtime performance for this platform is key.
Reply with the commands in markdown.

System information:
{system_info}

Rust toolchain information:
{rust_info}
"""

response = openai.chat.completions.create(model=models[0], messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))

You do not need to install anything. A full Rust toolchain is already installed and active:
- rustc 1.92.0 (stable)
- cargo 1.92.0
- rustup 1.28.2
- Target: x86_64-unknown-linux-gnu

For a single-file program main.rs with maximum runtime performance (compile time can be slow), use rustc directly with aggressive optimization flags:

Python subprocess commands:
```python
compile_command = [
    "/home/sanjif/.cargo/bin/rustc",
    "-C", "opt-level=3",
    "-C", "lto=fat",
    "-C", "codegen-units=1",
    "-C", "target-cpu=native",
    "-C", "panic=abort",
    "-C", "overflow-checks=off",
    "main.rs",
    "-o", "main",
]

run_command = ["./main"]
```

Notes:
- target-cpu=native enables AVX2/FMA and CPU-specific tuning for your Ryzen 9, maximizing performance on this machine (binaries may not be portable to older CPUs).
- lto=fat and codegen-units=1 favor runtime speed over compile time.
- panic=abort removes unwinding overhead; if you need to catch panics, omit this flag.
- This assumes main.rs is self-contained (no external crates). If it uses crates, compile with cargo instead.

## For C++, overwrite this with the commands from yesterday, or for Rust, use the new commands

Or just use the website like yesterday:

 https://www.programiz.com/cpp-programming/online-compiler/

In [14]:
compile_command = [
    "/home/sanjif/.cargo/bin/rustc",
    "-C", "opt-level=3",
    "-C", "lto=fat",
    "-C", "codegen-units=1",
    "-C", "target-cpu=native",
    "-C", "panic=abort",
    "-C", "overflow-checks=off",
    "main.rs",
    "-o", "main",
]

run_command = ["./main"]


## And now, on with the main task

In [15]:
language = "Rust" # or "C++"
extension = "rs" if language == "Rust" else "cpp"

system_prompt = f"""
Your task is to convert Python code into high performance {language} code.
Respond only with {language} code. Do not provide any explanation other than occasional comments.
The {language} response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to {language} with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.{language} and then compiled and executed; the compilation command is:
{compile_command}
Respond only with {language} code.
Python code to port:

```python
{python}
```
"""

In [16]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [17]:
def write_output(code):
    with open(f"main.{extension}", "w") as f:
        f.write(code)

In [18]:
def port(model, python):
    client = clients[model]
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```rust','').replace('```','')
    return reply

In [19]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [20]:
# Use the commands from GPT 5

def compile_and_run(code):
    write_output(code)
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [21]:
python_hard = """# Be careful to support large numbers

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
from styles import CSS

with gr.Blocks(css=CSS, theme=gr.themes.Monochrome(), title=f"Port from Python to {language}") as ui:
    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python = gr.Code(
                label="Python (original)",
                value=python_hard,
                language="python",
                lines=26
            )
        with gr.Column(scale=6):
            cpp = gr.Code(
                label=f"{language} (generated)",
                value="",
                language="cpp",
                lines=26
            )

    with gr.Row(elem_classes=["controls"]):
        python_run = gr.Button("Run Python", elem_classes=["run-btn", "py"])
        model = gr.Dropdown(models, value=models[0], show_label=False)
        convert = gr.Button(f"Port to {language}", elem_classes=["convert-btn"])
        cpp_run = gr.Button(f"Run {language}", elem_classes=["run-btn", "cpp"])

    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python_out = gr.TextArea(label="Python result", lines=8, elem_classes=["py-out"])
        with gr.Column(scale=6):
            cpp_out = gr.TextArea(label=f"{language} result", lines=8, elem_classes=["cpp-out"])

    convert.click(fn=port, inputs=[model, python], outputs=[cpp])
    python_run.click(fn=run_python, inputs=[python], outputs=[python_out])
    cpp_run.click(fn=compile_and_run, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Opening in existing browser session.


## RESULTS!

Qwen 2.5 Coder: FAIL  
Gemini 2.5 Pro: FAIL  
DeepSeek Coder v2: FAIL  
Qwen3 Coder 30B: FAIL  
Claude Sonnet 4.5: FAIL    
GPT-5: FAIL    

3rd place: GPT-oss-20B: 0.000341  
2nd place: Grok 4: 0.000317  
**1st place: OpenAI GPT-OSS 120B: 0.000304**  

In [23]:
print(f"In Ed's experimenet, the GPT-OSS 120B model outcome is {33.755209/0.000304:,.0f} times faster than the Python code.")